## Amazon Scraping

In this notebook, I am presenting a procedure for collecting product information from the first seven pages of [Amazon.ca](https://amazon.ca/) after searching a custom keyword. The program is able to collect:
1. Product Title
2. Price
3. Number of Reviews
4. Review Rating
5. url address

The program saves the results in a csv file, named "Results".

In [1]:
import csv
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager



# get_url defines the url address for a specific seach-term on a specific page number on Amazon

def get_url (search_term,page_num):
    S= list(search_term.lower().split())
    seaterm=S[0]
    for i in S[1:]:
        seaterm=seaterm+'+'+i
    return 'https://www.amazon.ca/s?k={}&page={}&crid=2ODRV04KOTK3K&sprefix={}%2Caps%2C148&ref=nb_sb_noss'.format(seaterm,page_num,
                                                                                                                  seaterm)



driver = webdriver.Chrome(ChromeDriverManager().install())

# amazon_scraping2 is the function that scrape data for a specific search term until page 7 of the results.
# item title, price, number of review, review rate, and the url address are being saved. 


def amazon_scraping (search_term):
    
    title = []
    price = []
    review = []
    item = []
    url = []
    num_of_review=[]
    m=0
    
    for j in range (0,7):
        
        url_search = get_url(search_term,j+1)
        driver.get(url_search)    
        soup = BeautifulSoup (driver.page_source,'html.parser')

        for i in range(0,200):
            try:
                
                results2 = soup.find_all('span',{'cel_widget_id':'MAIN-SEARCH_RESULTS-{}'.format(i)})
                if not results2:
                    results2 = soup.find_all('div',{'cel_widget_id':'MAIN-SEARCH_RESULTS-{}'.format(i)})

                title.append(results2[0].find('h2').text)
                url.append('https://www.amazon.ca'+results2[0].h2.a.get('href'))
                
                try:
                    price.append(results2[0].find('span',{'class':'a-price', 'data-a-color':'base'}).text.split('$')[-1])
                    
                except AttributeError:
                    price.append('')
                try:
                    review.append (results2[0].find('i','a-icon-star-small').text)
                    if (results2[0].find_all('span',{'class':'a-size-base'})[0].text.replace(',','')).isdecimal():
                        num_of_review.append ((results2[0].find_all('span',{'class':'a-size-base'})[0].text.replace(',','')))
                    else:
                        num_of_review.append('')
                    ## num_of_review.append ((results2[0].find_all('span',{'class':'a-size-base'})[1].text.replace(',','')))
                except AttributeError:
                    review.append('')
                    num_of_review.append('')

            except IndexError: 
                m+=1
                
        
    

        intermediate_dictionary = {'Title':title[0:len(review)], 'Price':price[0:len(review)], 
                                   'NumberReview': num_of_review [0:len(review)],'Review':review[0:len(review)], 'url':url[0:len(review)]}
                                                                                                            
        Results=pd.DataFrame(intermediate_dictionary)
        
   # results are being saved in a csv file named Results.      
    Results.to_csv('Results.csv')


    return





====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\hamed\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [10]:
amazon_scraping('music box')

In [11]:
Result = pd.read_csv('Results.csv')
Result.head(10)

,Unnamed: 0,Title,Price,NumberReview,Review,url
0,0,ROKR DIY Music Box Kit a Lovely Song-Wind Up M...,25.99,78.0,4.5 out of 5 stars,https://www.amazon.ca/gp/slredirect/picassoRed...
1,1,Pirates of The Caribbean Music Box Hand Crank ...,17.46,500.0,4.3 out of 5 stars,https://www.amazon.ca/gp/slredirect/picassoRed...
2,2,3D Crystal Ball Music Box Luminous Rotating Mu...,43.00,195.0,4.4 out of 5 stars,https://www.amazon.ca/gp/slredirect/picassoRed...
3,3,Crystal Ball Music Box Mechanical music box Wo...,38.90,6.0,4.6 out of 5 stars,https://www.amazon.ca/gp/slredirect/picassoRed...
4,4,"Over The Rainbow Music Boxes,Vintage Wood Hand...",14.99,46.0,4.1 out of 5 stars,https://www.amazon.ca/Rainbow-Vintage-Birthday...
5,5,"You are My Sunshine Wood Music Boxes,Laser Eng...",17.99,4471.0,4.1 out of 5 stars,https://www.amazon.ca/Sunshine-Engraved-Birthd...
6,6,"Wooden Music Box, Crystal Ball Music Box,Mecha...",42.98,30.0,4.8 out of 5 stars,https://www.amazon.ca/Mechanical-Wonderful-Swe...
7,7,"Number-One Wooden Hand Crank Music Box, You ar...",15.99,144.0,4.2 out of 5 stars,https://www.amazon.ca/Number-One-Sunshine-Clas...
8,8,"Antique Engraved Wooden Wind-Up Musical Box,Yo...",23.99,141.0,4.6 out of 5 stars,https://www.amazon.ca/Engraved-Sunshine-Gold-p...
9,9,Wood Music Boxes- La Vie En Rose Carved Hand C...,16.99,344.0,4.7 out of 5 stars,https://www.amazon.ca/Musical-Handmade-Engrave...
